# Can we predict if the outcome of the board's decision will favor the complainant?

Sunwoo Kim

## Summary of Findings


### Introduction
For this project, I have built the classifier model that determines whether the outcome of the case favored comlaints or police by looking at several different features. Therefore, my classification will be binary classification where 1 in result means it favored complaint and 0 means it did not favored the complaint. To have that binary result, I took the 'board_desposition' column and mapped to binary value based on the value that the column holds. To predict the outcome, we can use almost every column's value since they are all recorded when the incident was happened and reported with the complain, but any columns with the data regarding about what happened after(date closed) cannot be considered as out features. I chose accuracy as my metrics since there is no major downside of picking false negative but false postive or false negative are equally weighted.

### Baseline Model
2 features that I used for my baseline model is 'fado_type' and 'rank_incident.' Fado type is 'Force, abuse of authoruty, Dscourtesy, and Offensive language' and I thought this is a relevant feature since the type of abuse of can change the severity of the case which can affect the outcome. Also, I used 'rank_incident', rank of officer at the time of incident' as my feature. I believe this feature was relevant since the how much authority the officer has can affect the decision. 'fado_type' is nominal and 'rank_incident' is ordinal. For those 2 features, I used one hot encoding.
I think the performance of my base model is not bad but there is definitely a room to improve on.

### Final Model
First, I decided to add two new feature created by me. One feature is about whether the year complain was recieved was under Democratic power or Republican power. I thought this was relevant to the decision since two parties most likely have two difference stance on police officer's authority. The difference in mean for democratic and republican era is pretty significant as well. Also, I checked if the complainant ethnicity was whether white or POC since I believed there is systematic discrimination and that is also present through the mean difference. I converted it to binary value showing whether they are white or not. Also, I ran throug each combinations of hyperparameter and with each combination, I ran 10 test and measure mean accruacy to determine which combination is the best and it was max_depth=10, min_samples_split=15, and criterion='gini'. To see if my model was improved compared to my baseline model, I generated 20 different random test split and calculated accuracy for baseline model and my final model and for all those 20 cases, my final model had better accuracy which shows the improvement. 

### Fairness Analysis
I wanted to see if my model perfor differntly based on the fact whether they are senior or not. 
Null Hypothesis: Fianl model has no differnece in performance whether the complainant was a senior or not. 
Alternative Hypotehsis: Fianl model performs differently based on the fact that whether complainant was a senior or not.
We will use 0.05 as our p-value.

To test this out, I calculated precision for the time when complainant age was lower than 60 and above 60 and calculated the differnce. Then, I shuffled the data and generated new test and training datasets for 200 times and recorded the differnece in precision. Lastly, I calculated the percentage of time when the difference in precision was bigger than our acutal final model performance. As a result, we got a p-value of 0.545 which is bigger than 0.05. 
In conclusion, we can conclude there is no sufficient evidence to say my final model's performance vary based on complainant's age and that my final model performs uniformly.

## Code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import QuantileTransformer
from sklearn.neighbors import KNeighborsClassifier

In [3]:
alleg = pd.read_csv('allegations_202007271729.csv')
alleg = alleg.dropna()
alleg

,unique_mos_id,first_name,last_name,command_now,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,...,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition
0,10004,Jonathan,Ruiz,078 PCT,8409,42835,7,2019,5,2020,...,32,Black,Female,38.0,Abuse of Authority,Failure to provide RTKA card,78.0,Report-domestic dispute,No arrest made or summons issued,Substantiated (Command Lvl Instructions)
1,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,...,24,Black,Male,26.0,Discourtesy,Action,67.0,Moving violation,Moving violation summons issued,Substantiated (Charges)
2,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,...,24,Black,Male,26.0,Offensive Language,Race,67.0,Moving violation,Moving violation summons issued,Substantiated (Charges)
3,10007,John,Sears,078 PCT,5952,26146,7,2012,9,2013,...,25,Black,Male,45.0,Abuse of Authority,Question,67.0,PD suspected C/V of violation/crime - street,No arrest made or summons issued,Substantiated (Charges)
5,10012,Paula,Smith,078 PCT,4021,37256,5,2017,10,2017,...,50,White,Male,31.0,Abuse of Authority,Refusal to process civilian complaint,78.0,C/V telephoned PCT,No arrest made or summons issued,Substantiated (Command Lvl Instructions)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33353,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,36,Asian,Male,21.0,Discourtesy,Word,66.0,Moving violation,Moving violation summons issued,Unsubstantiated
33354,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,36,Asian,Male,21.0,Abuse of Authority,Interference with recording,66.0,Moving violation,Moving violation summons issued,Unsubstantiated
33355,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,36,Asian,Male,21.0,Abuse of Authority,Search (of person),66.0,Moving violation,Moving violation summons issued,Substantiated (Formalized Training)
33356,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,36,Asian,Male,21.0,Abuse of Authority,Vehicle search,66.0,Moving violation,Moving violation summons issued,Substantiated (Formalized Training)


In [4]:
alleg.columns

Index(['unique_mos_id', 'first_name', 'last_name', 'command_now', 'shield_no',
       'complaint_id', 'month_received', 'year_received', 'month_closed',
       'year_closed', 'command_at_incident', 'rank_abbrev_incident',
       'rank_abbrev_now', 'rank_now', 'rank_incident', 'mos_ethnicity',
       'mos_gender', 'mos_age_incident', 'complainant_ethnicity',
       'complainant_gender', 'complainant_age_incident', 'fado_type',
       'allegation', 'precinct', 'contact_reason', 'outcome_description',
       'board_disposition'],
      dtype='object')

### Baseline Model

In [6]:
#mapping the board disposition to binary value
result = {'Substantiated (Command Lvl Instructions)':1,
       'Substantiated (Charges)':1, 'Substantiated (Command Discipline A)':1,
       'Substantiated (Formalized Training)':1, 'Unsubstantiated':0,
       'Exonerated':0, 'Substantiated (Command Discipline B)':1,
       'Substantiated (Instructions)':1,
       'Substantiated (Command Discipline)':1,
       'Substantiated (No Recommendations)':1,
       'Substantiated (MOS Unidentified)':1}
alleg['result']=alleg['board_disposition'].map(result)

In [7]:
preproc = ColumnTransformer(transformers =[('features',OneHotEncoder(handle_unknown = 'ignore'), ['fado_type','rank_incident'])])
out=[]
#run the tes 20 time and get the average accuracy
for i in range(20):
    X_train, X_test, y_train, y_test = \
    train_test_split(alleg[['rank_incident','fado_type']],alleg['result'],test_size=0.25)
    BaseLine = Pipeline([('proc',preproc),('Fit',DecisionTreeClassifier())])
    BaseLine.fit(X_train,y_train)
    out.append(np.mean(BaseLine.predict(X_test)==y_test))

In [8]:
#baseline model accuracy
np.mean(out)

0.7545908896187349

### Final Model

In [9]:
a = alleg[alleg['complainant_ethnicity']=='White']
b = alleg[alleg['complainant_ethnicity']!='White']
print(b['result'].mean())
a['result'].mean()

0.2425296442687747


0.28023598820059

In [10]:
def det(x):
    dem = range(1993,2001)
    dem2 = range(2009,2017)
    if x in dem or x in dem2:
        return 1
    return 0
a = alleg.copy()
a['det'] = a['year_received'].apply(det)
a.groupby('det')['result'].mean()

det
0    0.230682
1    0.260095
Name: result, dtype: float64

In [11]:
#determines the administrative during that time
def det(col):
    dem = range(1993,2001)
    dem2 = range(2009,2017)
    def helper(x):
        if x in dem or x in dem2:
            return 1
        return 0
    return pd.DataFrame(col[col.columns[0]].apply(helper))
def ethBi(col):
    return pd.DataFrame(col[col.columns[0]]=='White')

preproc = ColumnTransformer(transformers =[('features',OneHotEncoder(handle_unknown = 'ignore'), ['fado_type','rank_incident','allegation']),\
                                           ('party', FunctionTransformer(det), ['year_received']),\
                                            ('eth', FunctionTransformer(ethBi), ['complainant_ethnicity'])])
FinalModel = Pipeline([('proc',preproc),\
                       ('Fit',DecisionTreeClassifier(max_depth=10,min_samples_split=15,criterion='gini'))])

In [16]:
#find the best hyperparameter
out2={}
depth = [2, 3, 4, 5, 7, 10, 13, 15, 18, None]
split = [2, 3, 5, 7, 10, 15, 20]
crit = ['gini', 'entropy']
for i in depth:
    for j in split:
        for k in crit:
            temp=[]
            for n in range(10):
                FinalModel = Pipeline([('proc',preproc),\
                                       ('Fit',DecisionTreeClassifier(max_depth=i,min_samples_split=j,criterion=k))])
                X_train, X_test, y_train, y_test = \
                train_test_split(alleg.drop(['result'],axis=1),alleg['result'],test_size=0.25)
                FinalModel.fit(X_train,y_train)
                temp.append(np.mean(FinalModel.predict(X_test)==y_test))
            out2[(i,j,k)]=np.mean(temp)  
out2

{(2, 2, 'gini'): 0.7574896472940169,
 (2, 2, 'entropy'): 0.7582036270169927,
 (2, 3, 'gini'): 0.7580893902613166,
 (2, 3, 'entropy'): 0.7620448379266028,
 (2, 5, 'gini'): 0.7618734827930888,
 (2, 5, 'entropy'): 0.7575610452663144,
 (2, 7, 'gini'): 0.7564329573040125,
 (2, 7, 'entropy'): 0.7576752820219906,
 (2, 10, 'gini'): 0.7568185063544195,
 (2, 10, 'entropy'): 0.758017992289019,
 (2, 15, 'gini'): 0.7597886620019991,
 (2, 15, 'entropy'): 0.7599885763244323,
 (2, 20, 'gini'): 0.7609024703698416,
 (2, 20, 'entropy'): 0.7584035413394259,
 (3, 2, 'gini'): 0.7572611737826647,
 (3, 2, 'entropy'): 0.7594887905183493,
 (3, 3, 'gini'): 0.7583749821505069,
 (3, 3, 'entropy'): 0.7597315436241611,
 (3, 5, 'gini'): 0.7576752820219907,
 (3, 5, 'entropy'): 0.7577752391832071,
 (3, 7, 'gini'): 0.7603312865914609,
 (3, 7, 'entropy'): 0.7569898614879338,
 (3, 10, 'gini'): 0.7588890475510496,
 (3, 10, 'entropy'): 0.7589318863344282,
 (3, 15, 'gini'): 0.7588604883621305,
 (3, 15, 'entropy'): 0.75760388

In [17]:
max(out2,key=out2.get)

(10, 15, 'gini')

In [18]:
out2[(10, 15, 'gini')]

0.7734827930886763

In [61]:
#run 20 differnet test split to see if my final model perforemd better than my baseline model all the time
out=[]
for i in range(20):
    X_train, X_test, y_train, y_test = \
    train_test_split(alleg.drop(['result'],axis=1),alleg['result'],test_size=0.25)
    FinalModel.fit(X_train,y_train)
    BaseLine.fit(X_train,y_train)
    a = np.mean(FinalModel.predict(X_test)==y_test)>np.mean(BaseLine.predict(X_test)==y_test)
    out.append(a)
a.all()

True

### Fairness Analysis

In [49]:
def prec(x,y):
    tp = sum((x==y)&(x==1))
    fp = sum((x!=y)&(x==1))
    return tp/(tp+fp)

In [62]:
X_train, X_test, y_train, y_test = \
    train_test_split(alleg.drop(['result'],axis=1),alleg['result'],test_size=0.25)
FinalModel.fit(X_train,y_train)
a = np.array(X_test['complainant_age_incident']<60)
pred1 = FinalModel.predict(X_test[a])
prec1 = prec(pred1,np.array(y_test[a]))
pred2 = FinalModel.predict(X_test[a==False])
prec2 = prec(pred2,np.array(y_test[a==False]))
test_stat=abs(prec1-prec2)
print(prec1,prec2,test_stat)

0.5959183673469388 0.5714285714285714 0.02448979591836742


In [64]:
out=[]
temp = alleg.copy()
for i in range(200):
    temp['shuffled'] = temp['complainant_age_incident'].sample(frac=1).reset_index(drop=True)
    X_train, X_test, y_train, y_test = \
    train_test_split(temp.drop(['result'],axis=1),temp['result'],test_size=0.25)
    a = np.array(X_test['shuffled']<60)
    ##test case with age less than 60
    pred1 = FinalModel.predict(X_test[a])
    prec1 = prec(pred1,np.array(y_test[a]))
    ##test case with age over than 60
    pred2 = FinalModel.predict(X_test[a==False])
    prec2 = prec(pred2,np.array(y_test[a==False]))
    b = abs(prec1-prec2)
    out.append(b)

In [65]:
#p-value
np.mean(np.array(out)>= test_stat)

0.545